# Light Gradient-Boosting Machine (LightGBM)

## Import Libraries and Data Loading

In [55]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, precision_score
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
import pickle
import lightgbm as lgb

train_df_raw = pd.read_feather('../data/processed/train_dataset.feather')
val_df_raw = pd.read_feather('../data/processed/validation_dataset.feather')
test_df_raw = pd.read_feather('../data/processed/test_dataset.feather')

## Drop unneccessary columns

In [56]:
train_df = train_df_raw.drop(['index','qid1','qid2'],axis=1)
val_df = val_df_raw.drop(['index','qid1','qid2'],axis=1)
test_df = test_df_raw.drop(['index','qid1','qid2'],axis=1)
train_df.head(5)

,question1,question2,q1_cleaned,q2_cleaned,q1_trimmed,q2_trimmed,q1_start,q2_start,q1_topic,q2_topic,...,same_starting,same_ending,wmdistance,dist_cosine,dist_cityblock,dist_canberra,dist_euclidean,dist_minkowski,is_duplicate,bert_prob_0
0,Has Ancient Chaldea been scientifically tested?,Has Ancient History been scientifically tested...,ha ancient chaldea been scientifically tested,ha ancient history been scientifically tested ...,ha ancient chaldea been scientifically tested,ha ancient history been scientifically tested ...,ha,ha,Free/Ocatopm/Software/Website,Relationship/Girl/Guy/People/Life,...,1,0,0.192389,0.238296,34.632359,167.718247,2.658521,2.658521,0,0.412712
1,What makes someone a good teacher?,What should be the first and most important qu...,what make someone a good teacher,what should be the first and most important qu...,what make someone a good teacher,what should be the first and most important qu...,what,what,Social Media/Gadget/Email,English/Law/Writing,...,1,1,0.343090,0.097663,24.251799,145.539053,1.837212,1.837212,1,0.210485
2,Which is the best state for education for a in...,What is the best state in the U.S for internat...,which is the best state for education for a in...,what is the best state in the united state for...,which is the best state for education for a in...,what is the best state in the united state for...,which,what,Purpose/Energy,Job/College/University,...,0,0,0.098867,0.015203,9.078124,89.634960,0.659214,0.659214,1,0.153916
3,What are the most common ridge patterns in fin...,Which ridge pattern is the least common in fin...,what are the most common ridge pattern in fing...,which ridge pattern is the least common in fin...,what are the most common ridge pattern in fing...,which ridge pattern is the least common in fin...,what,which,Language/Relationship,Language/Relationship,...,0,1,0.129774,0.039147,13.554628,110.654286,0.986643,0.986643,0,0.906878
4,Social History: When did Americans start givin...,Today is my nephew 6th birthday. What is the b...,social history when did american start giving ...,today is my nephew 6th birthday what is the be...,social history when did american start giving ...,today is my nephew 6th birthday what is the be...,social,today,Self-help/Learn/Business,Job/College/University,...,0,0,0.230667,0.115745,25.036501,168.859774,1.817673,1.817673,0,0.999865


### Here, we take only columns that are not string, hence omitting questions, topics, starting words, etc.

In [57]:
X_train, X_val, X_test = train_df.select_dtypes(include=np.number).drop('is_duplicate',axis = 1), val_df.select_dtypes(include=np.number).drop('is_duplicate', axis =1), test_df.select_dtypes(include=np.number).drop('is_duplicate', axis =1)
y_train, y_val = train_df[['is_duplicate']], val_df[['is_duplicate']]

In [58]:
X_train.shape
y_train.shape

(323429, 25)

(323429, 1)

In [59]:
X_train.head(5)

,length_diff,same_question,lc_substring,lc_subsequence,jaccard_dist,common_words,common_ratio,levenshtein,fuzz_qratio,fuzz_wratio,...,same_topic,same_starting,same_ending,wmdistance,dist_cosine,dist_cityblock,dist_canberra,dist_euclidean,dist_minkowski,bert_prob_0
0,-95,0,27,39,0.219697,5,0.027473,0.428571,43,86,...,0,1,0,0.192389,0.238296,34.632359,167.718247,2.658521,2.658521,0.412712
1,-36,0,15,27,0.148148,4,0.039604,0.534653,53,86,...,0,1,1,0.343090,0.097663,24.251799,145.539053,1.837212,1.837212,0.210485
2,13,0,22,52,0.494382,9,0.060403,0.697987,77,90,...,0,0,0,0.098867,0.015203,9.078124,89.634960,0.659214,0.659214,0.153916
3,0,0,16,34,0.455882,6,0.056075,0.635514,64,81,...,1,0,1,0.129774,0.039147,13.554628,110.654286,0.986643,0.986643,0.906878
4,-7,0,10,27,0.062500,2,0.015748,0.425197,43,51,...,0,0,0,0.230667,0.115745,25.036501,168.859774,1.817673,1.817673,0.999865


In [60]:
X_train.describe()

,length_diff,same_question,lc_substring,lc_subsequence,jaccard_dist,common_words,common_ratio,levenshtein,fuzz_qratio,fuzz_wratio,...,same_topic,same_starting,same_ending,wmdistance,dist_cosine,dist_cityblock,dist_canberra,dist_euclidean,dist_minkowski,bert_prob_0
count,323429.000000,323429.0,323429.000000,323429.000000,323429.000000,323429.000000,323429.000000,323429.000000,323429.000000,323429.000000,...,323429.000000,323429.000000,323429.000000,3.234290e+05,323429.000000,323429.000000,323429.000000,323429.000000,323429.000000,323429.000000
mean,-0.509824,0.0,18.301099,34.451104,0.283078,5.062957,0.047562,0.620982,61.946223,76.511210,...,0.477765,0.494501,0.285806,inf,0.093800,20.409918,131.646328,1.505318,1.505318,0.639239
std,32.680816,0.0,14.550269,16.761430,0.227183,3.085492,0.025286,0.183955,18.376092,15.286261,...,0.499506,0.499971,0.451798,NaN,0.081258,8.979431,36.493995,0.670176,0.670176,0.425000
min,-1080.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000812
25%,-11.000000,0.0,10.000000,24.000000,0.106599,3.000000,0.028571,0.467290,47.000000,66.000000,...,0.000000,0.000000,0.000000,1.613421e-01,0.040284,14.751558,110.957854,1.082978,1.082978,0.112546
50%,0.000000,0.0,15.000000,31.000000,0.225000,5.000000,0.046512,0.608696,61.000000,85.000000,...,0.000000,0.000000,0.000000,2.195602e-01,0.073056,19.610207,134.248227,1.441646,1.441646,0.950934
75%,12.000000,0.0,22.000000,41.000000,0.410959,7.000000,0.066667,0.767123,76.000000,86.000000,...,1.000000,1.000000,1.000000,2.815077e-01,0.121864,25.196007,155.439775,1.858509,1.858509,0.999382
max,487.000000,0.0,221.000000,280.000000,1.000000,41.000000,0.200000,1.000000,100.000000,100.000000,...,1.000000,1.000000,1.000000,inf,1.032621,118.019440,300.000000,8.676815,8.676815,0.999884


## Data Processing

As earlier found, there are some infinity values caused by divison error. We proceed to replace them with 0.

In [61]:
X_train.replace([np.inf, -np.inf], 0, inplace=True)
X_val.replace([np.inf, -np.inf], 0, inplace=True)
X_test.replace([np.inf, -np.inf], 0, inplace=True)

In [62]:
columns_to_scale = ['length_diff','jaccard_dist','common_ratio','levenshtein','fuzz_qratio','fuzz_wratio','wmdistance','dist_cosine','dist_cityblock',
                   'dist_canberra','dist_euclidean','dist_minkowski']

scaler = StandardScaler()

X_train.loc[:,columns_to_scale] = scaler.fit_transform(X_train.loc[:,columns_to_scale])
X_val.loc[:,columns_to_scale] = scaler.transform(X_val.loc[:,columns_to_scale])
X_test.loc[:,columns_to_scale] = scaler.transform(X_test.loc[:,columns_to_scale])

## 1. Building Base LightGBM

In [63]:
lgb_params = {"boosting": "gbdt", 'learning_rate': 0.05,
 "feature_fraction": 0.6, "bagging_freq": 1, "bagging_fraction": 0.8 , 'n_estimators': 100000,
 "metric": 'mae', "lambda_l1": 0.1, 'num_leaves': 32, 'min_data_in_leaf': 50, "num_threads": 8,
 "bagging_seed" : 1024,
 "seed": 1024,
 'feature_fraction_seed': 1024,
 }

In [64]:
clf = lgb.LGBMClassifier(**lgb_params)
clf.fit(X_train, y_train, eval_set=[(X_train, y_train),(X_val, y_val)], verbose = 1, eval_metric ='logloss',early_stopping_rounds=200)     

C:\Users\Calven\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Calven\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Calven\anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\Calven\anaconda3\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' ar

[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] lambda_l1 is set=0.1, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.1
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] num_threads is set=8, n_jobs=-1 will be ignored. Current value: num_threads=8
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[1]	training's binary_logloss: 0.617363	training's l1: 0.446079	valid_1's binary_logloss: 0.617394	valid_1's l1: 0.446076
[2]	training's binary_logloss: 0.580554	training's l1: 0.427401	valid_1's binary_logloss: 0.58060

[65]	training's binary_logloss: 0.1683	training's l1: 0.127999	valid_1's binary_logloss: 0.168986	valid_1's l1: 0.128268
[66]	training's binary_logloss: 0.165563	training's l1: 0.125182	valid_1's binary_logloss: 0.166272	valid_1's l1: 0.125465
[67]	training's binary_logloss: 0.162961	training's l1: 0.122496	valid_1's binary_logloss: 0.163662	valid_1's l1: 0.122783
[68]	training's binary_logloss: 0.160509	training's l1: 0.119947	valid_1's binary_logloss: 0.161225	valid_1's l1: 0.12024
[69]	training's binary_logloss: 0.158231	training's l1: 0.117524	valid_1's binary_logloss: 0.158933	valid_1's l1: 0.117816
[70]	training's binary_logloss: 0.158028	training's l1: 0.117292	valid_1's binary_logloss: 0.158756	valid_1's l1: 0.117593
[71]	training's binary_logloss: 0.157819	training's l1: 0.117056	valid_1's binary_logloss: 0.158576	valid_1's l1: 0.117368
[72]	training's binary_logloss: 0.155678	training's l1: 0.114772	valid_1's binary_logloss: 0.156439	valid_1's l1: 0.115096
[73]	training's bin

[141]	training's binary_logloss: 0.120885	training's l1: 0.0735334	valid_1's binary_logloss: 0.122835	valid_1's l1: 0.0742966
[142]	training's binary_logloss: 0.120838	training's l1: 0.0735208	valid_1's binary_logloss: 0.122814	valid_1's l1: 0.0742919
[143]	training's binary_logloss: 0.120686	training's l1: 0.0732883	valid_1's binary_logloss: 0.122667	valid_1's l1: 0.0740634
[144]	training's binary_logloss: 0.120538	training's l1: 0.0730645	valid_1's binary_logloss: 0.122538	valid_1's l1: 0.073847
[145]	training's binary_logloss: 0.120451	training's l1: 0.0730027	valid_1's binary_logloss: 0.122475	valid_1's l1: 0.073795
[146]	training's binary_logloss: 0.120395	training's l1: 0.0729889	valid_1's binary_logloss: 0.122445	valid_1's l1: 0.0737846
[147]	training's binary_logloss: 0.120245	training's l1: 0.0727777	valid_1's binary_logloss: 0.122325	valid_1's l1: 0.073574
[148]	training's binary_logloss: 0.120195	training's l1: 0.0727681	valid_1's binary_logloss: 0.122315	valid_1's l1: 0.073

[212]	training's binary_logloss: 0.115911	training's l1: 0.0684277	valid_1's binary_logloss: 0.119982	valid_1's l1: 0.0696584
[213]	training's binary_logloss: 0.11586	training's l1: 0.0683951	valid_1's binary_logloss: 0.119962	valid_1's l1: 0.069631
[214]	training's binary_logloss: 0.115796	training's l1: 0.0683652	valid_1's binary_logloss: 0.119955	valid_1's l1: 0.0696032
[215]	training's binary_logloss: 0.115751	training's l1: 0.0683263	valid_1's binary_logloss: 0.119949	valid_1's l1: 0.0695681
[216]	training's binary_logloss: 0.115703	training's l1: 0.0683039	valid_1's binary_logloss: 0.119916	valid_1's l1: 0.0695536
[217]	training's binary_logloss: 0.115668	training's l1: 0.0682974	valid_1's binary_logloss: 0.119916	valid_1's l1: 0.0695575
[218]	training's binary_logloss: 0.115625	training's l1: 0.068271	valid_1's binary_logloss: 0.119901	valid_1's l1: 0.0695399
[219]	training's binary_logloss: 0.115575	training's l1: 0.0682432	valid_1's binary_logloss: 0.119873	valid_1's l1: 0.069

[286]	training's binary_logloss: 0.112974	training's l1: 0.0670205	valid_1's binary_logloss: 0.119624	valid_1's l1: 0.0687525
[287]	training's binary_logloss: 0.112945	training's l1: 0.0670106	valid_1's binary_logloss: 0.119618	valid_1's l1: 0.0687496
[288]	training's binary_logloss: 0.112913	training's l1: 0.0669959	valid_1's binary_logloss: 0.11961	valid_1's l1: 0.0687388
[289]	training's binary_logloss: 0.112873	training's l1: 0.0669855	valid_1's binary_logloss: 0.119603	valid_1's l1: 0.0687354
[290]	training's binary_logloss: 0.112837	training's l1: 0.0669705	valid_1's binary_logloss: 0.119598	valid_1's l1: 0.0687296
[291]	training's binary_logloss: 0.112803	training's l1: 0.0669471	valid_1's binary_logloss: 0.119594	valid_1's l1: 0.0687116
[292]	training's binary_logloss: 0.112766	training's l1: 0.0669179	valid_1's binary_logloss: 0.119576	valid_1's l1: 0.0686921
[293]	training's binary_logloss: 0.112734	training's l1: 0.0669105	valid_1's binary_logloss: 0.119581	valid_1's l1: 0.0

[364]	training's binary_logloss: 0.110582	training's l1: 0.0661052	valid_1's binary_logloss: 0.119511	valid_1's l1: 0.068381
[365]	training's binary_logloss: 0.110552	training's l1: 0.0660938	valid_1's binary_logloss: 0.119516	valid_1's l1: 0.068378
[366]	training's binary_logloss: 0.110517	training's l1: 0.0660723	valid_1's binary_logloss: 0.119516	valid_1's l1: 0.0683646
[367]	training's binary_logloss: 0.110487	training's l1: 0.0660674	valid_1's binary_logloss: 0.119523	valid_1's l1: 0.0683696
[368]	training's binary_logloss: 0.110456	training's l1: 0.0660587	valid_1's binary_logloss: 0.119519	valid_1's l1: 0.0683705
[369]	training's binary_logloss: 0.11043	training's l1: 0.0660444	valid_1's binary_logloss: 0.119514	valid_1's l1: 0.0683608
[370]	training's binary_logloss: 0.110396	training's l1: 0.0660364	valid_1's binary_logloss: 0.119503	valid_1's l1: 0.0683595
[371]	training's binary_logloss: 0.110373	training's l1: 0.0660335	valid_1's binary_logloss: 0.119505	valid_1's l1: 0.068

[430]	training's binary_logloss: 0.108924	training's l1: 0.0655977	valid_1's binary_logloss: 0.1198	valid_1's l1: 0.068331
[431]	training's binary_logloss: 0.108894	training's l1: 0.0655848	valid_1's binary_logloss: 0.119791	valid_1's l1: 0.0683293
[432]	training's binary_logloss: 0.108857	training's l1: 0.0655649	valid_1's binary_logloss: 0.119774	valid_1's l1: 0.0683171
[433]	training's binary_logloss: 0.10883	training's l1: 0.0655565	valid_1's binary_logloss: 0.119773	valid_1's l1: 0.0683179
[434]	training's binary_logloss: 0.108792	training's l1: 0.0655427	valid_1's binary_logloss: 0.119748	valid_1's l1: 0.0683099
[435]	training's binary_logloss: 0.10876	training's l1: 0.0655291	valid_1's binary_logloss: 0.119749	valid_1's l1: 0.068308
[436]	training's binary_logloss: 0.108727	training's l1: 0.0655197	valid_1's binary_logloss: 0.119747	valid_1's l1: 0.0683103
[437]	training's binary_logloss: 0.10869	training's l1: 0.0655071	valid_1's binary_logloss: 0.119739	valid_1's l1: 0.0683047

[504]	training's binary_logloss: 0.106859	training's l1: 0.0647958	valid_1's binary_logloss: 0.119565	valid_1's l1: 0.0680445
[505]	training's binary_logloss: 0.106833	training's l1: 0.0647879	valid_1's binary_logloss: 0.119556	valid_1's l1: 0.0680395
[506]	training's binary_logloss: 0.10681	training's l1: 0.0647735	valid_1's binary_logloss: 0.119543	valid_1's l1: 0.0680278
[507]	training's binary_logloss: 0.106784	training's l1: 0.0647669	valid_1's binary_logloss: 0.119549	valid_1's l1: 0.0680274
[508]	training's binary_logloss: 0.106754	training's l1: 0.0647592	valid_1's binary_logloss: 0.119556	valid_1's l1: 0.0680285
[509]	training's binary_logloss: 0.106719	training's l1: 0.0647379	valid_1's binary_logloss: 0.119559	valid_1's l1: 0.0680138
[510]	training's binary_logloss: 0.106694	training's l1: 0.0647244	valid_1's binary_logloss: 0.119549	valid_1's l1: 0.0680047
[511]	training's binary_logloss: 0.10667	training's l1: 0.0647257	valid_1's binary_logloss: 0.119544	valid_1's l1: 0.06

LGBMClassifier(bagging_fraction=0.8, bagging_freq=1, bagging_seed=1024,
               boosting='gbdt', feature_fraction=0.6,
               feature_fraction_seed=1024, lambda_l1=0.1, learning_rate=0.05,
               metric='mae', min_data_in_leaf=50, n_estimators=100000,
               num_leaves=32, num_threads=8, seed=1024)

In [92]:
preds_train = clf.predict(X_train)
preds_prob_train = clf.predict_proba(X_train)
preds_val = clf.predict(X_val)
preds_prob_val = clf.predict_proba(X_val)

In [66]:
print("The train log loss is:", log_loss(y_train, preds_prob_train))
print("The train precision is:", precision_score(y_train, preds_train))
print("The validation log loss is:", log_loss(y_val, preds_prob_val))
print("The validation precision is:", precision_score(y_val, preds_val))

The train log loss is: 0.11092481595205556
The train precision is: 0.9342233826801652
The validation log loss is: 0.1194613686613149
The validation precision is: 0.9332585899887885


#### Get predicted probablity for test set for second layer stacking
- Here, we only want the predicted probability of one class, hence we will only fetch the second column, which is predicted probability of being duplicated.

In [67]:
preds_prob_test = clf.predict_proba(X_test)[:, 1]

## 2. Refine with more features from TFIDF
- Now we will try with the dataset with vectorized features

In [68]:
train_df_raw = pd.read_feather('../data/processed/x_train_dataset.feather')
val_df_raw = pd.read_feather('../data/processed/x_validation_dataset.feather')
test_df_raw = pd.read_feather('../data/processed/x_test_dataset.feather')

In [69]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    train_df_raw.head(5)

,index,question1,question2,q1_start,q2_start,q1_topic,q2_topic,length_diff,same_question,lc_substring,lc_subsequence,jaccard_dist,common_words,common_ratio,levenshtein,fuzz_qratio,fuzz_wratio,q2_question_mark_count,q1_question_mark_count,question_mark_count_diff,freq_q1+q2,freq_q1-q2,same_topic,same_starting,same_ending,wmdistance,dist_cosine,dist_cityblock,dist_canberra,dist_euclidean,dist_minkowski,q1_cleaned_v2,q2_cleaned_v2,q1_vector,q2_vector,q1_vec_0,q1_vec_1,q1_vec_2,q1_vec_3,q1_vec_4,q2_vec_0,q2_vec_1,q2_vec_2,q2_vec_3,q2_vec_4,q1_Assessment/Word/Home,q1_Best/Way/Visit,q1_Country/Car/Show/Television,q1_Engine/Password/Search,q1_English/Law/Writing,q1_Food/Health,q1_Free/Ocatopm/Software/Website,q1_India/Government/China,q1_Interview/Difference/Drug,q1_Job/College/University,q1_Language/Relationship,q1_Long/Review/Work/Compare,q1_Lose/Weight/Time/Travel/Salary,q1_Money/Bank/Online,q1_Movie/Video Game/Youtube,q1_Politics/Trump/Election,q1_Problem/Increase,q1_Purpose/Energy,q1_Quora/Question/Google/Answer,q1_Relationship/Girl/Guy/People/Life,q1_Self-help/Learn/Business,q1_Sex/Woman/Man,q1_Social Media/Gadget/Email,q1_United States/Day,q1_Year/New/Stock/Company,q2_Assessment/Word/Home,q2_Best/Way/Visit,q2_Country/Car/Show/Television,q2_Engine/Password/Search,q2_English/Law/Writing,q2_Food/Health,q2_Free/Ocatopm/Software/Website,q2_India/Government/China,q2_Interview/Difference/Drug,q2_Job/College/University,q2_Language/Relationship,q2_Long/Review/Work/Compare,q2_Lose/Weight/Time/Travel/Salary,q2_Money/Bank/Online,q2_Movie/Video Game/Youtube,q2_Politics/Trump/Election,q2_Problem/Increase,q2_Purpose/Energy,q2_Quora/Question/Google/Answer,q2_Relationship/Girl/Guy/People/Life,q2_Self-help/Learn/Business,q2_Sex/Woman/Man,q2_Social Media/Gadget/Email,q2_United States/Day,q2_Year/New/Stock/Company,is_duplicate
0,33525,Has Ancient Chaldea been scientifically tested?,Has Ancient History been scientifically tested...,ha,ha,Free/Ocatopm/Software/Website,Relationship/Girl/Guy/People/Life,-95,0,27,39,0.219697,5,0.027473,0.428571,43,86,4,1,-3,11,-9,0,1,0,0.192389,0.238296,34.632359,167.718247,2.658521,2.658521,ha ancient chaldea scientifically tested,ha ancient history scientifically tested real ...,"[0.031408057, 0.10707768, 0.32202494, -0.23581...","[0.14967845, 0.2725273, -0.03749804, -0.142553...",0.031408,0.107078,0.322025,-0.235812,0.065765,0.149678,0.272527,-0.037498,-0.142554,0.114115,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,369925,What makes someone a good teacher?,What should be the first and most important qu...,what,what,Social Media/Gadget/Email,English/Law/Writing,-36,0,15,27,0.148148,4,0.039604,0.534653,53,86,1,1,0,5,-3,0,1,1,0.343090,0.097663,24.251799,145.539053,1.837212,1.837212,make someone good teacher,first important quality good teacher,"[-0.005436562, 0.03173224, 0.027050504, 0.0323...","[0.008946306, 0.057755146, 0.05656157, 0.01483...",-0.005437,0.031732,0.027051,0.032324,0.018609,0.008946,0.057755,0.056562,0.014834,0.035993,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,176858,Which is the best state for education for a in...,What is the best state in the U.S for internat...,which,what,Purpose/Energy,Job/College/University,13,0,22,52,0.494382,9,0.060403,0.697987,77,90,1,1,0,2,0,0,0,0,0.098867,0.015203,9.078124,89.634960,0.659214,0.659214,best state education international student uni...,best state united state international student,"[0.17608762, 0.09855839, 0.28985745, -0.232633...","[0.11724252, 0.14971006, 0.29985344, -0.303659...",0.176088,0.098558,0.289857,-0.232634,-0.038018,0.117243,0.149710,0.299853,-0.303660,-0.060581,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,293958,What are the most common ridge patterns in fin...,Which ridge pattern is the least common in fin...,what,which,Language/Relationship,Language/Relationship,0,0,16

In [70]:
train_df = train_df_raw.drop(['index','q1_start','q2_start','q1_topic','q2_topic','question1','question2','q1_cleaned_v2','q2_cleaned_v2'],axis=1)
val_df = val_df_raw.drop(['index','q1_start','q2_start','q1_topic','q2_topic','question1','question2','q1_cleaned_v2','q2_cleaned_v2'],axis=1)
test_df = test_df_raw.drop(['index','q1_start','q2_start','q1_topic','q2_topic','question1','question2','q1_cleaned_v2','q2_cleaned_v2'],axis=1)

In [71]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    train_df.head(5)
train_df['same_question'].unique()
train_df['same_topic'].unique()

,length_diff,same_question,lc_substring,lc_subsequence,jaccard_dist,common_words,common_ratio,levenshtein,fuzz_qratio,fuzz_wratio,q2_question_mark_count,q1_question_mark_count,question_mark_count_diff,freq_q1+q2,freq_q1-q2,same_topic,same_starting,same_ending,wmdistance,dist_cosine,dist_cityblock,dist_canberra,dist_euclidean,dist_minkowski,q1_vector,q2_vector,q1_vec_0,q1_vec_1,q1_vec_2,q1_vec_3,q1_vec_4,q2_vec_0,q2_vec_1,q2_vec_2,q2_vec_3,q2_vec_4,q1_Assessment/Word/Home,q1_Best/Way/Visit,q1_Country/Car/Show/Television,q1_Engine/Password/Search,q1_English/Law/Writing,q1_Food/Health,q1_Free/Ocatopm/Software/Website,q1_India/Government/China,q1_Interview/Difference/Drug,q1_Job/College/University,q1_Language/Relationship,q1_Long/Review/Work/Compare,q1_Lose/Weight/Time/Travel/Salary,q1_Money/Bank/Online,q1_Movie/Video Game/Youtube,q1_Politics/Trump/Election,q1_Problem/Increase,q1_Purpose/Energy,q1_Quora/Question/Google/Answer,q1_Relationship/Girl/Guy/People/Life,q1_Self-help/Learn/Business,q1_Sex/Woman/Man,q1_Social Media/Gadget/Email,q1_United States/Day,q1_Year/New/Stock/Company,q2_Assessment/Word/Home,q2_Best/Way/Visit,q2_Country/Car/Show/Television,q2_Engine/Password/Search,q2_English/Law/Writing,q2_Food/Health,q2_Free/Ocatopm/Software/Website,q2_India/Government/China,q2_Interview/Difference/Drug,q2_Job/College/University,q2_Language/Relationship,q2_Long/Review/Work/Compare,q2_Lose/Weight/Time/Travel/Salary,q2_Money/Bank/Online,q2_Movie/Video Game/Youtube,q2_Politics/Trump/Election,q2_Problem/Increase,q2_Purpose/Energy,q2_Quora/Question/Google/Answer,q2_Relationship/Girl/Guy/People/Life,q2_Self-help/Learn/Business,q2_Sex/Woman/Man,q2_Social Media/Gadget/Email,q2_United States/Day,q2_Year/New/Stock/Company,is_duplicate
0,-95,0,27,39,0.219697,5,0.027473,0.428571,43,86,4,1,-3,11,-9,0,1,0,0.192389,0.238296,34.632359,167.718247,2.658521,2.658521,"[0.031408057, 0.10707768, 0.32202494, -0.23581...","[0.14967845, 0.2725273, -0.03749804, -0.142553...",0.031408,0.107078,0.322025,-0.235812,0.065765,0.149678,0.272527,-0.037498,-0.142554,0.114115,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,-36,0,15,27,0.148148,4,0.039604,0.534653,53,86,1,1,0,5,-3,0,1,1,0.343090,0.097663,24.251799,145.539053,1.837212,1.837212,"[-0.005436562, 0.03173224, 0.027050504, 0.0323...","[0.008946306, 0.057755146, 0.05656157, 0.01483...",-0.005437,0.031732,0.027051,0.032324,0.018609,0.008946,0.057755,0.056562,0.014834,0.035993,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,13,0,22,52,0.494382,9,0.060403,0.697987,77,90,1,1,0,2,0,0,0,0,0.098867,0.015203,9.078124,89.634960,0.659214,0.659214,"[0.17608762, 0.09855839, 0.28985745, -0.232633...","[0.11724252, 0.14971006, 0.29985344, -0.303659...",0.176088,0.098558,0.289857,-0.232634,-0.038018,0.117243,0.149710,0.299853,-0.303660,-0.060581,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,0,0,16,34,0.455882,6,0.056075,0.635514,64,81,1,1,0,2,0,1,0,1,0.129774,0.039147,13.554628,110.654286,0.986643,0.986643,"[0.25171572, -0.024394771, 0.11217328, -0.0639...","[0.29362243, 0.050006576, 0.08588519, -0.15508...",0.251716,-0.024395,0.112173,-0.063949,-0.128590,0.293622,0.050007,0.085885,-0.155080,-0.202898,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,-7,0,10,27,0.062500,2,0.015748,0.425197,43,51,1,1,0,2,0,0,0,0,0.230667,0.115745,25.036501,168.859774,1.817673,1.817673,"[0.09621305, 0.1520336, 0.030658064, 0.1090956...","[-0.21848702, 0.02774278, -0.096341625, 0.3227...",0.096213,0.152034,0.030658,0.109096,0.195408,-0.218487,0.027743,-0.096342,0.322707,0.008506,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


array([0])

array([0, 1])

### Data Preparation
- Scaling
- Filtering

In [72]:
train_df = train_df.select_dtypes(include=np.number)
val_df = val_df.select_dtypes(include=np.number)
test_df = test_df.select_dtypes(include=np.number)

In [73]:
column_names = train_df.columns

In [74]:
train_df['same_question'].unique()
train_df['same_topic'].unique()

array([0])

array([0, 1])

In [75]:
train_df.replace([np.inf, -np.inf], 0, inplace=True)
val_df.replace([np.inf, -np.inf], 0, inplace=True)
test_df.replace([np.inf, -np.inf], 0, inplace=True)

columns_to_scale = ['length_diff','jaccard_dist','common_ratio','levenshtein','fuzz_qratio','fuzz_wratio','wmdistance','dist_cosine','dist_cityblock',
                   'dist_canberra','dist_euclidean','dist_minkowski']

scaler = StandardScaler()
    

train_df.loc[:,columns_to_scale] = scaler.fit_transform(train_df.loc[:,columns_to_scale])
val_df.loc[:,columns_to_scale] = scaler.transform(val_df.loc[:,columns_to_scale])
test_df.loc[:,columns_to_scale] = scaler.transform(test_df.loc[:,columns_to_scale])


In [76]:
train_df = pd.DataFrame(train_df, columns=column_names)
val_df =  pd.DataFrame(val_df, columns=column_names)
test_df =  pd.DataFrame(test_df, columns=column_names)

In [77]:
train_df['is_duplicate'] = train_df['is_duplicate'].astype(int)
val_df['is_duplicate'] = val_df['is_duplicate'].astype(int)
test_df['is_duplicate'] = test_df['is_duplicate'].astype(int)

In [78]:
predictors = train_df.loc[:, train_df.columns != 'is_duplicate'].columns.tolist()
target = 'is_duplicate'
categorical_features = ['same_question','same_topic','same_starting','same_ending']

In [79]:
train_df['same_question'].unique()
train_df['same_topic'].unique()
train_df.head(5)

array([0])

array([0, 1])

,length_diff,same_question,lc_substring,lc_subsequence,jaccard_dist,common_words,common_ratio,levenshtein,fuzz_qratio,fuzz_wratio,...,q2_Problem/Increase,q2_Purpose/Energy,q2_Quora/Question/Google/Answer,q2_Relationship/Girl/Guy/People/Life,q2_Self-help/Learn/Business,q2_Sex/Woman/Man,q2_Social Media/Gadget/Email,q2_United States/Day,q2_Year/New/Stock/Company,is_duplicate
0,-2.891308,0,27,39,-0.278987,5,-0.794481,-1.045966,-1.031027,0.620741,...,0,0,0,1,0,0,0,0,0,0
1,-1.085965,0,15,27,-0.593926,4,-0.314716,-0.469292,-0.486841,0.620741,...,0,0,0,0,0,0,0,0,0,1
2,0.413388,0,22,52,0.930104,9,0.507816,0.418606,0.819206,0.882414,...,0,0,0,0,0,0,0,0,0,1
3,0.015600,0,16,34,0.760639,6,0.336659,0.078998,0.111764,0.293649,...,0,0,0,0,0,0,0,0,0,0
4,-0.198593,0,10,27,-0.970927,2,-1.258153,-1.064311,-1.031027,-1.668901,...,0,0,0,0,0,0,0,0,0,0


In [80]:
def lgb_modelfit_nocv(params, dtrain, dvalid, predictors, target='target', objective='binary', metrics='binary_logloss',
                 feval=None, early_stopping=20, num_boost_round=3000, categorical_features=None):
    lgb_params = {
        'boosting_type': 'gbdt',
        'objective': objective,
        'metric':metrics,
        'n_estimators': 100000,
        'lambda_l1': 0.1, 
        'bagging_seed' : 1024,
        'categorical_feature' : categorical_features
    }

    lgb_params.update(params)

    lgbtrain= lgb.Dataset(dtrain[predictors].values, label=dtrain[target].values,
                          feature_name=predictors,
                          categorical_feature=categorical_features
                          )
    lgbval = lgb.Dataset(dvalid[predictors].values, label=dvalid[target].values,
                          feature_name=predictors,
                          categorical_feature=categorical_features
                          )

    evals_results = {}

    bst = lgb.train(lgb_params, 
                     lgbtrain, 
                     valid_sets=[lgbtrain, lgbval], 
                     valid_names=['train','valid'], 
                     evals_result=evals_results, 
                     num_boost_round=num_boost_round,
                     early_stopping_rounds = early_stopping,
                     feval=feval)

    n_estimators = bst.best_iteration
    print("\nModel Report")
    print("n_estimators : ", n_estimators)
    for metric in metrics:
        print(metric+":", evals_results['valid'][metric][n_estimators-1])
    
    return bst

In [81]:
params = {
    'learning_rate': 0.05,
    'num_leaves': 7,  # 2^max_depth - 1
    'max_depth': 3,  # -1 means no limit
    'min_child_samples': 100,  # Minimum number of data need in a child(min_data_in_leaf)
    'max_bin': 100,  # Number of bucketed bin for feature values
    'scale_pos_weight':0.360 # because training data is unbalanced 
}
bst = lgb_modelfit_nocv(params, 
                        train_df, 
                        val_df, 
                        predictors, 
                        target, 
                        objective='binary',
                        metrics=['binary_logloss','average_precision'],
                        early_stopping=30, 
                        categorical_features = categorical_features)

C:\Users\Calven\anaconda3\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
C:\Users\Calven\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\Calven\anaconda3\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
C:\Users\Calven\anaconda3\lib\site-packages\lightgbm\engine.py:260: UserWarning: 'evals_result' argument is deprecated and will be removed in a future release of LightGBM. Pass 'record_evaluation()' callback via 'callbacks' argument ins

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] categorical_feature is set=same_question,same_topic,same_starting,same_ending, categorical_column=1,15,16,17 will be ignored. Current value: categorical_feature=same_question,same_topic,same_starting,same_ending
[LightGBM] [Info] Number of positive: 119410, number of negative: 204019
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.029491 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2690
[LightGBM] [Info] Number of data points in the train set: 323429, number of used features: 83
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.369200 -> initscore=-0.535650
[LightGBM] [Info] Start training from score -0.535650
[1]	train's binary_logloss: 0.647678	train's average_precision: 0.673146	valid's binary_logloss: 0.647588	v

C:\Users\Calven\anaconda3\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
C:\Users\Calven\anaconda3\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[5]	train's binary_logloss: 0.612574	train's average_precision: 0.716263	valid's binary_logloss: 0.612165	valid's average_precision: 0.718058
[6]	train's binary_logloss: 0.605425	train's average_precision: 0.725147	valid's binary_logloss: 0.604911	valid's average_precision: 0.727264
[7]	train's binary_logloss: 0.598932	train's average_precision: 0.727127	valid's binary_logloss: 0.598349	valid's average_precision: 0.729199
[8]	train's binary_logloss: 0.593121	train's average_precision: 0.741294	valid's binary_logloss: 0.592436	valid's average_precision: 0.743525
[9]	train's binary_logloss: 0.587822	train's average_precision: 0.743312	valid's binary_logloss: 0.587135	valid's average_precision: 0.745037
[10]	train's binary_logloss: 0.583135	train's average_precision: 0.745408	valid's binary_logloss: 0.582396	valid's average_precision: 0.747273
[11]	train's binary_logloss: 0.578721	train's average_precision: 0.746081	valid's binary_logloss: 0.577966	valid's average_precision: 0.747598
[12]

[65]	train's binary_logloss: 0.508241	train's average_precision: 0.792933	valid's binary_logloss: 0.50717	valid's average_precision: 0.794333
[66]	train's binary_logloss: 0.507788	train's average_precision: 0.793281	valid's binary_logloss: 0.50668	valid's average_precision: 0.794782
[67]	train's binary_logloss: 0.507386	train's average_precision: 0.793592	valid's binary_logloss: 0.506321	valid's average_precision: 0.795065
[68]	train's binary_logloss: 0.506777	train's average_precision: 0.794072	valid's binary_logloss: 0.505769	valid's average_precision: 0.795573
[69]	train's binary_logloss: 0.506461	train's average_precision: 0.794357	valid's binary_logloss: 0.505494	valid's average_precision: 0.795823
[70]	train's binary_logloss: 0.50589	train's average_precision: 0.794884	valid's binary_logloss: 0.504922	valid's average_precision: 0.796072
[71]	train's binary_logloss: 0.505184	train's average_precision: 0.795362	valid's binary_logloss: 0.504247	valid's average_precision: 0.796447
[7

...

[4585]	train's binary_logloss: 0.39405	train's average_precision: 0.878843	valid's binary_logloss: 0.411949	valid's average_precision: 0.864146
[4586]	train's binary_logloss: 0.394045	train's average_precision: 0.878846	valid's binary_logloss: 0.411947	valid's average_precision: 0.864147
[4587]	train's binary_logloss: 0.394041	train's average_precision: 0.878849	valid's binary_logloss: 0.411945	valid's average_precision: 0.864149
[4588]	train's binary_logloss: 0.394034	train's average_precision: 0.878854	valid's binary_logloss: 0.41195	valid's average_precision: 0.864147
[4589]	train's binary_logloss: 0.39403	train's average_precision: 0.878857	valid's binary_logloss: 0.411951	valid's average_precision: 0.864145
[4590]	train's binary_logloss: 0.394024	train's average_precision: 0.87886	valid's binary_logloss: 0.411951	valid's average_precision: 0.864145
[4591]	train's binary_logloss: 0.39402	train's average_precision: 0.878864	valid's binary_logloss: 0.41195	valid's average_precision: 

[4644]	train's binary_logloss: 0.393724	train's average_precision: 0.879064	valid's binary_logloss: 0.411872	valid's average_precision: 0.864182
[4645]	train's binary_logloss: 0.393719	train's average_precision: 0.879068	valid's binary_logloss: 0.411869	valid's average_precision: 0.864183
[4646]	train's binary_logloss: 0.393715	train's average_precision: 0.879071	valid's binary_logloss: 0.411866	valid's average_precision: 0.864187
[4647]	train's binary_logloss: 0.393708	train's average_precision: 0.879076	valid's binary_logloss: 0.411859	valid's average_precision: 0.864188
[4648]	train's binary_logloss: 0.3937	train's average_precision: 0.879082	valid's binary_logloss: 0.411858	valid's average_precision: 0.864188
[4649]	train's binary_logloss: 0.393695	train's average_precision: 0.879085	valid's binary_logloss: 0.411854	valid's average_precision: 0.86419
[4650]	train's binary_logloss: 0.393689	train's average_precision: 0.87909	valid's binary_logloss: 0.411849	valid's average_precision

[4701]	train's binary_logloss: 0.393447	train's average_precision: 0.879255	valid's binary_logloss: 0.411775	valid's average_precision: 0.864222
[4702]	train's binary_logloss: 0.393442	train's average_precision: 0.879259	valid's binary_logloss: 0.411773	valid's average_precision: 0.864222
[4703]	train's binary_logloss: 0.393436	train's average_precision: 0.879262	valid's binary_logloss: 0.411775	valid's average_precision: 0.86422
[4704]	train's binary_logloss: 0.393429	train's average_precision: 0.879265	valid's binary_logloss: 0.411773	valid's average_precision: 0.864222
[4705]	train's binary_logloss: 0.393424	train's average_precision: 0.879268	valid's binary_logloss: 0.411772	valid's average_precision: 0.864221
[4706]	train's binary_logloss: 0.393417	train's average_precision: 0.879273	valid's binary_logloss: 0.411763	valid's average_precision: 0.864227
[4707]	train's binary_logloss: 0.39341	train's average_precision: 0.879279	valid's binary_logloss: 0.411759	valid's average_precisi

[4761]	train's binary_logloss: 0.393148	train's average_precision: 0.879453	valid's binary_logloss: 0.411682	valid's average_precision: 0.864256
[4762]	train's binary_logloss: 0.393141	train's average_precision: 0.879459	valid's binary_logloss: 0.411673	valid's average_precision: 0.864262
[4763]	train's binary_logloss: 0.393136	train's average_precision: 0.879461	valid's binary_logloss: 0.411673	valid's average_precision: 0.864262
[4764]	train's binary_logloss: 0.393132	train's average_precision: 0.879465	valid's binary_logloss: 0.41167	valid's average_precision: 0.864266
[4765]	train's binary_logloss: 0.393127	train's average_precision: 0.879471	valid's binary_logloss: 0.411664	valid's average_precision: 0.864269
[4766]	train's binary_logloss: 0.393121	train's average_precision: 0.879475	valid's binary_logloss: 0.411667	valid's average_precision: 0.864268
[4767]	train's binary_logloss: 0.393115	train's average_precision: 0.879478	valid's binary_logloss: 0.411667	valid's average_precis

[4818]	train's binary_logloss: 0.392812	train's average_precision: 0.879699	valid's binary_logloss: 0.411558	valid's average_precision: 0.864314
[4819]	train's binary_logloss: 0.392807	train's average_precision: 0.879703	valid's binary_logloss: 0.411562	valid's average_precision: 0.864309
[4820]	train's binary_logloss: 0.392801	train's average_precision: 0.879707	valid's binary_logloss: 0.411568	valid's average_precision: 0.864302
[4821]	train's binary_logloss: 0.392795	train's average_precision: 0.879712	valid's binary_logloss: 0.411567	valid's average_precision: 0.864302
[4822]	train's binary_logloss: 0.392789	train's average_precision: 0.879716	valid's binary_logloss: 0.411567	valid's average_precision: 0.864301
[4823]	train's binary_logloss: 0.392785	train's average_precision: 0.879718	valid's binary_logloss: 0.411563	valid's average_precision: 0.864304
[4824]	train's binary_logloss: 0.392779	train's average_precision: 0.879723	valid's binary_logloss: 0.411557	valid's average_preci

Build Predictions

In [82]:
preds_prob_train2 = bst.predict(train_df.drop(["is_duplicate"], axis=1), num_iteration=bst.best_iteration)
preds_prob_val2 = bst.predict(val_df.drop(["is_duplicate"], axis=1), num_iteration=bst.best_iteration)
print("The train log loss is:", log_loss(y_train, preds_prob_train2))
print("The validation log loss is:", log_loss(y_val, preds_prob_val2))


preds_prob_test2 = bst.predict(test_df.drop(["is_duplicate"], axis=1), num_iteration=bst.best_iteration)

The train log loss is: 0.3926922301868105
The validation log loss is: 0.41152569558955826


### We can see that based on this model, the log loss and precision performs slightly worse than the previous vanilla model.
- We will proceed to perform bagging; taking the bagged of these two boosted trees.
<br>Bagged Boosted Trees:

    1. Predict with each N
    2. Average the predictions to get the final prediction


#### We first evaluate the bagged boosted trees with validation.

In [91]:
preds_prob_val_final = (0.5 * preds_prob_val[:, 1]) + (0.5 * preds_prob_val2)
print("The bagged validation log loss is:", log_loss(y_val, preds_prob_val_final))

preds_val_final = [1 if n >= 0.5 else 0 for n in preds_prob_val_final]
print("The bagged validation precision is:", precision_score(y_val, preds_val_final ))

The bagged validation log loss is: 0.20539651589152416
The bagged validation precision is: 0.9539632584351899


#### We can see that there is a slight increase in log loss but a much significant gain in precision.

### We now proceed to build the features for the final model
#### We standardise all predicted probabilities to be the predicted probabilities of class 0, since these values are the probabilities of class 1 (duplicated), we subtract them from 1 and output as the feature for the final model.

In [93]:
preds_prob_train = 1 - preds_prob_train[:, 1] # First Boosted Tree
preds_prob_train2 = 1 - preds_prob_train2 # Second Boosted Tree


preds_prob_test = 1 - preds_prob_test # First Boosted Tree
preds_prob_test2 = 1 - preds_prob_test2# Second Boosted Tree

In [94]:
preds_train_final = (0.5 * preds_prob_train) + (0.5 * preds_prob_train2)
preds_train_final


preds_test_final = (0.5 * preds_prob_test) + (0.5 * preds_prob_test2)
preds_test_final

array([0.85745791, 0.43762612, 0.34171523, ..., 0.85739481, 0.93460334,
       0.99927077])

array([0.97370239, 0.79672051, 0.99710103, ..., 0.94932581, 0.56722549,
       0.33803203])

#### Output predicted probabilities into pickle for final model

In [96]:
pickle.dump(preds_test_final, open("../data/processed/predictions_full_LGBM_test.p", "wb" ) )
np.savetxt("../data/processed/predictions_full_LGBM_train.csv", preds_train_final, delimiter=",")
np.savetxt("../data/processed/predictions_full_LGBM_test.csv", preds_test_final, delimiter=",")